In [ ]:
'''
    Author: Rajat Gupta
            rg321110@gmail.com
    LinkedIn: www.linkedin.com/in/rajat-gupta-4aab0324b
    Git Repo: https://github.com/Lavi2111/tokyo-olympic-azure-de-project
    Date: 04-03-2024
    Data Source: Kaggle
    Dashboard link: 
    *** The following code to be implemented on databricks. ***
'''

## Mounting Point

In [ ]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tanent_id/oauth2/token"}

In [ ]:
# Mount name for the bucket
mount_name ="/mnt/tokyoolympic"

# Container name
container = "tokyo-olympic-data"

# Storage account name
storageacc = "tokyoolympicdatarajat"

# Source url
SOURCE_URL = f"abfss://{container}@{storageacc}.dfs.core.windows.net" # container@storageacc

# Mount the drive
# dbutils.fs.mount(source=SOURCE_URL, mount_point=mount_name , extra_configs = configs)

# Unmount 
# dbutils.fs.unmount(mount_point=mount_name)

# source directory
source_dir = f"{mount_name}/raw-data/"
# target directory
target_dir = f"{mount_name}/transformed-data/"

### Checking mount point

In [ ]:
%fs 
ls "/mnt/tokyoolympic"

path,name,size,modificationTime
dbfs:/mnt/tokyoolympic/raw-data/,raw-data/,0,1709398862000
dbfs:/mnt/tokyoolympic/transformed-data/,transformed-data/,0,1709398874000


### Importing libraries

In [ ]:
# Importing necessary libraries
from pyspark.sql import functions as f
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType


### Dataframe

In [ ]:
file_type = 'csv'
athletes_df = spark \
            .read \
            .format(file_type) \
            .option("header","true") \
            .load(f"{source_dir}athletes.{file_type}")
coaches_df = spark \
            .read \
            .format(file_type) \
            .option("header","true") \
            .load(f"{source_dir}coaches.{file_type}")
entriesgender_df = spark \
            .read \
            .format(file_type) \
            .option("header","true") \
            .load(f"{source_dir}entriesgender.{file_type}")
medals_df = spark \
            .read \
            .format(file_type) \
            .option("header","true") \
            .load(f"{source_dir}medals.{file_type}")
teams_df = spark \
            .read \
            .format(file_type) \
            .option("header","true") \
            .load(f"{source_dir}teams.{file_type}")

In [ ]:
athletes_df.show()
athletes_df.printSchema()

+--------------------+--------------------+-------------------+
|                Name|                 NOC|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [ ]:
coaches_df.show()
coaches_df.printSchema()

+--------------------+--------------------+-----------------+--------+
|                Name|                 NOC|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    NULL|
|           ABE Junya|               Japan|       Volleyball|    NULL|
|       ABE Katsuhiko|               Japan|       Basketball|    NULL|
|        ADAMA Cherif|       Côte d'Ivoire|         Football|    NULL|
|          AGEBA Yuya|               Japan|       Volleyball|    NULL|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
|ALLER CARBALLO Ma...|               Spain|       Basketball|    NULL|
|     

In [ ]:
display(entriesgender_df)
entriesgender_df.printSchema()

Discipline,Female,Male,Total
3x3 Basketball,32,32,64
Archery,64,64,128
Artistic Gymnastics,98,98,196
Artistic Swimming,105,0,105
Athletics,969,1072,2041
Badminton,86,87,173
Baseball/Softball,90,144,234
Basketball,144,144,288
Beach Volleyball,48,48,96
Boxing,102,187,289


root
 |-- Discipline: string (nullable = true)
 |-- Female: string (nullable = true)
 |-- Male: string (nullable = true)
 |-- Total: string (nullable = true)



In [ ]:
# basic transformations on entriesgender_df
entriesgender_df = entriesgender_df \
                    .withColumn("Female",f.col("Female").cast(IntegerType())) \
                    .withColumn("Male",f.col("Male").cast(IntegerType())) \
                    .withColumn("Total",f.col("Total").cast(IntegerType()))

In [ ]:
entriesgender_df.show()
entriesgender_df.printSchema()

+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

In [ ]:
medals_df.show()
medals_df.printSchema()

+----+--------------------+----+------+------+-----+-------------+
|Rank|            Team/NOC|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [ ]:
# basic transformations on medals_df
medals_df = medals_df \
            .withColumn("Rank",f.col("Rank").cast(IntegerType())) \
            .withColumn("Gold",f.col("Gold").cast(IntegerType())) \
            .withColumn("Silver",f.col("Silver").cast(IntegerType())) \
            .withColumn("Bronze",f.col("Bronze").cast(IntegerType())) \
            .withColumn("Total",f.col("Total").cast(IntegerType())) \
            .withColumn("Rank by Total",f.col("Rank by Total").cast(IntegerType()))

In [ ]:
medals_df.show()
medals_df.printSchema()

+----+--------------------+----+------+------+-----+-------------+
|Rank|            Team/NOC|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [ ]:
teams_df.show()
teams_df.printSchema()

+-------------+--------------+--------------------+------------+
|         Name|    Discipline|                 NOC|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

# Some Analysis

## 1: Top 5 countries with highest number of medals

In [ ]:
top5_medal_countries = medals_df \
                              .select("Team/NOC","Total") \
                              .orderBy("Total",ascending=0) \
                              .withColumnRenamed("Total","Total Medals") \
                              .withColumnRenamed("Team/NOC","Country") \
                              .limit(5)
display(top5_medal_countries)

Country,Total Medals
United States of America,113
People's Republic of China,88
ROC,71
Great Britain,65
Japan,58


Databricks visualization. Run in Databricks to view.

## 2: Top 5 countries with highest number of gold medals

In [ ]:
top5_gold_medal_countries = medals_df \
                              .select("Team/NOC","Gold") \
                              .orderBy("Gold",ascending=0) \
                              .withColumnRenamed("Team/NOC","Country") \
                              .limit(5)
display(top5_gold_medal_countries)

Country,Gold
United States of America,39
People's Republic of China,38
Japan,27
Great Britain,22
ROC,20


Databricks visualization. Run in Databricks to view.

## 3: Top 5 countries with highest number of silver medals

In [ ]:
top5_silver_medal_countries = medals_df \
                              .select("Team/NOC","Silver") \
                              .orderBy("Silver",ascending=0) \
                              .withColumnRenamed("Team/NOC","Country") \
                              .limit(5)
display(top5_silver_medal_countries)

Country,Silver
United States of America,41
People's Republic of China,32
ROC,28
Great Britain,21
Japan,14


Databricks visualization. Run in Databricks to view.

## 4: Top 5 countries with highest number of bronze medals

In [ ]:
top5_bronze_medal_countries = medals_df \
                              .select("Team/NOC","Bronze") \
                              .orderBy("Bronze",ascending=0) \
                              .withColumnRenamed("Team/NOC","Country") \
                              .limit(5)
display(top5_bronze_medal_countries)

Country,Bronze
United States of America,33
ROC,23
Australia,22
Great Britain,22
Italy,20


Databricks visualization. Run in Databricks to view.

## 5: Calculate the average number of entries by gender for each discipline

In [ ]:
average_entries_by_gender = entriesgender_df.withColumn(
    'Avg_Female', entriesgender_df['Female'] / entriesgender_df['Total']
).withColumn(
    'Avg_Male', entriesgender_df['Male'] / entriesgender_df['Total']
)
average_entries_by_gender.show()

+--------------------+------+----+-----+-------------------+-------------------+
|          Discipline|Female|Male|Total|         Avg_Female|           Avg_Male|
+--------------------+------+----+-----+-------------------+-------------------+
|      3x3 Basketball|    32|  32|   64|                0.5|                0.5|
|             Archery|    64|  64|  128|                0.5|                0.5|
| Artistic Gymnastics|    98|  98|  196|                0.5|                0.5|
|   Artistic Swimming|   105|   0|  105|                1.0|                0.0|
|           Athletics|   969|1072| 2041| 0.4747672709456149| 0.5252327290543851|
|           Badminton|    86|  87|  173|0.49710982658959535| 0.5028901734104047|
|   Baseball/Softball|    90| 144|  234|0.38461538461538464| 0.6153846153846154|
|          Basketball|   144| 144|  288|                0.5|                0.5|
|    Beach Volleyball|    48|  48|   96|                0.5|                0.5|
|              Boxing|   102

# Loading data into target directory

In [ ]:
athletes_df \
    .repartition(1) \
    .write \
    .option("header","true") \
    .mode("overwrite") \
    .csv(f"{target_dir}athletes")
coaches_df \
    .repartition(1) \
    .write \
    .option("header","true") \
    .mode("overwrite") \
    .csv(f"{target_dir}coaches")
entriesgender_df \
    .repartition(1) \
    .write \
    .option("header","true") \
    .mode("overwrite") \
    .csv(f"{target_dir}entriesgender")
medals_df \
    .repartition(1) \
    .write \
    .option("header","true") \
    .mode("overwrite") \
    .csv(f"{target_dir}medals")
teams_df \
    .repartition(1) \
    .write \
    .option("header","true") \
    .mode("overwrite") \
    .csv(f"{target_dir}teams")